<a href="https://colab.research.google.com/github/33quitykubby/Rabbit_DNN_3/blob/main/Rabbit_RNN_2_Seq2Seq_RNN_wordPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2Seq (RNN)による単語予測

predict_word.ipynb  
をベースにSeq2Seqの単語予測  
写経


# 日本時間にする

In [ ]:
  !rm /etc/localtime
  !ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
  !date

Wed May  5 21:41:07 JST 2021


# マウント パス指定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_PATH = '/content/drive/MyDrive/RabbitChallenge/DNN_code_colab_lesson_3_4/lesson_3/3_2_tf_languagemodel/'

## 中間ファイルを削除

In [ ]:
import os
if ( os.path.isfile(BASE_PATH + "data_for_predict/word_dict.dic")):
  print("dicファイル削除")
  os.remove(BASE_PATH + "data_for_predict/word_dict.dic")

if ( os.path.isfile(BASE_PATH + "data_for_predict/predict_model.ckpt")):
  print("modelファイル削除")
  os.remove(BASE_PATH + "data_for_predict/predict_model.ckpt")


dicファイル削除


# ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import os
import glob

import librosa

import datetime

import gc

from sklearn.datasets import load_boston
from pandas import DataFrame

In [ ]:
#開始時刻
start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print("start_time=",start_time)

start_time= 2021-05-05 21:41:08.175636+09:00


# 乱数シードの初期化






In [ ]:
import os 
import numpy as np
from sklearn import model_selection
import tensorflow as tf
import random

# for Seq2Seq
import glob
import collections
import pickle
import re
import time

COMMON_SEED = 0
STANDARD_SEED = 0
NP_SEED = 0
TF_SEED = 0 

# np.random.seed(STANDARD_SEED)
# random.seed(NP_SEED)
# tf.random.set_seed(TF_SEED)

def seed_everything():
    random.seed(STANDARD_SEED)
    os.environ['PYTHONHASHSEED'] = str(COMMON_SEED)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    np.random.seed(NP_SEED)
    tf.random.set_seed(TF_SEED)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)


In [ ]:
#乱数シード固定

seed_everything()

# デバッグプリント

In [ ]:
DEBUG = True

def debugPrint(str, debug=True):
  if debug :
    print(str)

# Corpusクラス

In [ ]:
class Corpus:
  def __init__(self):
    self.unknown_word_symbol = "<???>"
    self.unknown_word_threshold = 3 # 未知誤と定義する単語の出現回数閾値
    self.corpus_file = BASE_PATH + "corpus/**/*/*.txt"
    self.corpus_encoding = "utf-8"
    self.dictionary = {}
    self.dictionary_filename = BASE_PATH + "data_for_predict/word_dict.dic"
    self.vocabulary_size = 0 #あとで挿入される

    self.chunk_size = 5
    if ( os.path.isfile(self.dictionary_filename)):
      self.load_dict()
    else :
      print("ファイルがない")

    words = []
    for filename in glob.glob(self.corpus_file, recursive=True):
      with open(filename, "r", encoding=self.corpus_encoding) as f:

        # word breaking
        text = f.read()
        #すべての文字を小文字に統一、開業をスペースに変換
        text = text.lower().replace("\n", " ")
        #特定の文字以外の文字を空文字に置換
        text = re.sub(r"[^a-z '\-]","",text)
        #複数のスペースはスペース一文字に置換
        text = re.sub(r"[ ]+"," ", text)

        #前処理:　'-'で始まる単語は無視
        words = [ word for word in text.split() if not word.startswith("-")]

    self.data_n = len(words) - self.chunk_size

    # build_dictしてからじゃないとsec_to_matrixがエラーになる
    self.build_dict()

    self.data = self.seq_to_matrix(words)

  def prepare_data(self):
    # 訓練データとテストデータを準備する
    # data_n = ( text データの総単語数 ) - chunk_size
    # input: (data_n, chunk_size vocabulary_size)
    # output: (data_n, vocabulary_size)

    print("prepare_data")

    # 入力と出力の次元テンソルを準備
    all_input = np.zeros([self.chunk_size, self.vocabulary_size, self.data_n])
    all_output = np.zeros([self.vocabulary_size, self.data_n])

    #準備したテンソルにコーパスのone-hot表現（self.data)のデータを埋める
    #i番目から(i+chunk_size-1)番目までの単語が1組の入力となる
    #この時の出力は(i+chunk_size)番目の単語
    for i in range(self.data_n):
        all_output[:, i] = self.data[:, i + self.chunk_size] # i + chunk_size番目の単語のone-hotベクトル
        for j in range(self.chunk_size):
          all_input[j, :, i] = self.data[:, i + self.chunk_size - j -1] # dataのi+chunk_size-j-1列のスライスをinputへ

    #のちに使うデータ形式に合わせるため転置を取る
    all_input = all_input.transpose([2, 0, 1])
    all_output = all_output.transpose()

    #訓練データ：テストデータを4:1に分割
    training_num = ( self.data_n * 4 ) // 5
    return all_input[:training_num], all_output[:training_num], all_input[training_num:], all_output[training_num:]

  def build_dict(self):
    # コーパス全体を見て単語の出現回数をカウント
    counter = collections.Counter()

    for filename in glob.glob(self.corpus_file, recursive=True):
      with open(filename, "r", encoding=self.corpus_encoding) as f:

        # word breaking
        text = f.read()
        #すべての文字を小文字に統一、開業をスペースに変換
        text = text.lower().replace("\n", " ")
        #特定の文字以外の文字を空文字に置換
        text = re.sub(r"[^a-z '\-]","",text)
        #複数のスペースはスペース一文字に置換
        text = re.sub(r"[ ]+"," ", text)

        #前処理:　'-'で始まる単語は無視
        words = [ word for word in text.split() if not word.startswith("-")]

        counter.update(words)

    # 出現頻度の低い単語を一つの記号にまとめる
    word_id = 0
    unknown_count = 0 #出現頻度が低い単語数
    dictionary = {}
    for word, count in counter.items():
      if count <= self.unknown_word_threshold:
        #出現頻度がthreshold以下ならcontinueされ、dictionaryに入らない
        unknown_count += 1
        continue

      
      dictionary[word] = word_id
      word_id +=1

    dictionary[self.unknown_word_symbol] = word_id

    print("総単語数=", word_id + unknown_count)
    print("うち、未知の単語数=", unknown_count)
    print("有効な単語数(unknown symbol含む)=", len(dictionary))

    #辞書をpickleで取っておく
    with open(self.dictionary_filename,"wb") as f:
      pickle.dump(dictionary,f)
      print("pickle保存済")
    
    self.dictionary = dictionary

    self.vocabulary_size = len(self.dictionary)
    self.input_layer_size = len(self.dictionary)
    self.output_layer_size = len(self.dictionary)

    print(self.dictionary)

  def load_dict(self):
    if os.path.getsize(self.dictionary_filename) > 0:
      with open(self.dictionary_filename, "rb") as f:
        self.dictionary = pickle.load(f)
        self.vocabulary_size = len(self.dictionary)
        self.input_layer_size = len(self.dictionary)
        self.output_layer_size = len(self.dictionary)
        print("有効な単語数=", self.input_layer_size)
    else:
      print("ファイルが空")

  def get_word_id(self, word):
    # print(word)
    return self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])

  def to_one_hot(self, word):
    index = self.get_word_id(word)
    data = np.zeros(self.vocabulary_size)
    data[index] = 1
    return data

  def seq_to_matrix(self, seq):
    print("seq=",seq)
    data = np.array([self.to_one_hot(word) for word in seq]) # (data_n, vocabulary_size)
    return data.transpose() # (vocabulrary_size, data_n)


## テスト

In [ ]:
cp = Corpus()


ファイルがない
総単語数= 34010
うち、未知の単語数= 24185
有効な単語数(unknown symbol含む)= 9826
pickle保存済
{'jet': 0, 'lag': 1, 'and': 2, 'the': 3, 'monday': 4, 'of': 5, 'my': 6, 'life': 7, 'i': 8, 'recently': 9, 'returned': 10, 'to': 11, 'us': 12, 'from': 13, 'australia': 14, 'flight': 15, 'took': 16, 'me': 17, 'morning': 18, 'in': 19, 'sydney': 20, 'again': 21, 'la': 22, 'crossing': 23, 'date': 24, 'line': 25, 'messed': 26, 'up': 27, 'sense': 28, 'time': 29, 'enough': 30, 'without': 31, 'added': 32, 'bonus': 33, 'thinking': 34, 'should': 35, 'be': 36, 'heading': 37, 'bed': 38, 'just': 39, 'as': 40, 'sun': 41, 'began': 42, 'climb': 43, 'into': 44, 'california': 45, 'sky': 46, 'you': 47, 'may': 48, 'familiar': 49, 'with': 50, 'concept': 51, 'name': 52, 'for': 53, 'circadian': 54, 'disruption': 55, 'sleep': 56, 'cycles': 57, 'rhythms': 58, 'if': 59, "you've": 60, 'had': 61, 'pleasure': 62, 'zones': 63, 'a': 64, 'plane': 65, 'whether': 66, 'it': 67, 'was': 68, 'mere': 69, 'hop': 70, 'one': 71, 'coast': 72, 'other': 

# Languageクラス

In [ ]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()

class Language:

  def __init__(self):
    print("init")
    self.corpus = Corpus()
    self.dictionary = self.corpus.dictionary
    self.vocabulary_size = len(self.dictionary)
    self.input_layer_size = self.vocabulary_size # 入力層数
    self.hidden_layer_size = 30 #隠れ層のRNNユニット数
    self.output_layer_size = self.vocabulary_size # 出力層数
    self.batch_size = 512
    self.chunk_size = 5 #展開するシーケンス数 c(0), c(1), ... ,c(chunk_size-1)と入力しc(chunk_size)番目の単語の確率が出力される
    self.learning_rate = 0.001
    # self.epochs = 1000
    self.epochs = 1000
    self.forget_bias = 1.0 # LSTM忘却ゲートのバイアス
    self.model_filename = BASE_PATH + "data_for_predict/predict_model.ckpt"
    self.unknown_word_symbol = self.corpus.unknown_word_symbol

  def inference(self, input_data, initial_state):
    print("inference")
    # param input(batch_size, chunk_size, vocabulary_size)
    # param initial_state(batch_size, hidden_size)

    # 重みとバイアスの初期化
    # tf.Variableで初期値変数を設定
    # truncated_normalは切断正規分布でstddev=0.01から外れるものは切断された分布を持つ変数値
    # www.mwsoft.jp/programming/tensor/tutorial_mnist.html
    hidden_w = tf.Variable(tf.compat.v1.truncated_normal([self.input_layer_size,self.hidden_layer_size], stddev=0.01))
    hidden_b = tf.Variable(tf.ones([self.hidden_layer_size]))

    output_w = tf.Variable(tf.compat.v1.truncated_normal([self.hidden_layer_size,self.output_layer_size], stddev=0.01))
    output_b = tf.Variable(tf.ones([self.output_layer_size]))
    
    # BasicLSTMCell, BasicRNNCellは(batch_size, hidden_layer_size)がchunk_size分つながったリストを入力する
    # 現時点での入力データは(batch_size, chunk_size, layer_size)という3次元テンソル
    # よって、tf.transposeやtf.reshapeなどによってテンソルサイズを調整する

    input_data = tf.transpose(input_data, [1, 0, 2]) # (chunk_size, batch_size, vocabulary_size)
    input_data = tf.reshape(input_data, [-1, self.input_layer_size]) # (chunk_size*batch_size, hidden_layer_size)
    input_data = tf.matmul(input_data, hidden_w) + hidden_b # 重みとバイアスBを適用 (chunk_size, batch_size)
    input_data = tf.split(input_data, self.chunk_size, 0) # リストに分割 chunk_size * (batch_size, hidden_layer_size)

    # RNNのセルを定義　(LSTM, GRUも使える)
    cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(self.hidden_layer_size)
    outputs, states = tf.compat.v1.nn.static_rnn(cell, input_data, initial_state=initial_state)

    # 最後に隠れ層から出力層につながる重みとバイアスの処理
    # 最終的にはsoftmaxで処理して確率になる
    # softmaxは混ん関数の外
    output = tf.matmul(outputs[-1], output_w) + output_b

    return output

  def loss (self, logits, labels):
    print("loss")
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

    return cost

  def training(self, cost):
    print("training")
    # 今回は最適化手法でAdamを選択
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(cost)

    return optimizer

  def train(self):
    print("train")
    #　変数準備
    input_data = tf.compat.v1.placeholder("float", [None, self.chunk_size, self.input_layer_size])
    actual_labels = tf.compat.v1.placeholder("float", [None, self.output_layer_size])
    initial_state = tf.compat.v1.placeholder("float", [None, self.hidden_layer_size])

    prediction = self.inference(input_data, initial_state)
    cost = self.loss(prediction, actual_labels)
    optimizer = self.training(cost)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(actual_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    # TensorBoardで可視化するためクロスエントロピーをサマリーに追加
    tf.summary.scalar("Cross entropy: ", cost)
    summary = tf.compat.v1.summary.merge_all()
    

    # 訓練、テストデータの用意
    trX, trY, teX, teY = self.corpus.prepare_data()
    # print(trX, trY, teX, teY )
    training_num = trX.shape[0]

    #ログを保存するためのディレクトリ
    timestamp = time.time()
    dirname = datetime.datetime.fromtimestamp(timestamp).strftime("%Y%m%d%H%M%S")

    #ここから実際の学習
    with tf.compat.v1.Session() as sess:
      sess.run(tf.compat.v1.global_variables_initializer())
      summary_writer = tf.compat.v1.summary.FileWriter(BASE_PATH + "log/" + dirname, sess.graph)

      #エポックを回す
      for epoch in range (self.epochs):
        step = 0
        epoch_loss = 0
        epoch_acc = 0

        #訓練データをバッチサイズごとに分けて学習させる
        #エポックごとの損失関数の合計値や精度も計算する
        while (step + 1)*self.batch_size < training_num:
          start_idx = step * self.batch_size
          end_idx = (step + 1) * self.batch_size

          batch_xs = trX[start_idx:end_idx, :, :]
          batch_ys = trY[start_idx:end_idx, :]

          _, c, a = sess.run([optimizer, cost, accuracy],
                             feed_dict = {input_data: batch_xs,
                                          actual_labels: batch_ys,
                                          initial_state: np.zeros([self.batch_size, self.hidden_layer_size])
                                          }
                            )
          epoch_loss += c
          epoch_acc += a
          step += 1

        #コンソールに損失関数の値や精度を出力しておく
        print("Epoch", epoch, "completed out of", self.epochs, "-- loss:", epoch_loss, "-- accuracy:", epoch_acc/step)

        # Epochが終わるごとにTensorBoard用に値を保存
        # print("summary:",summary)
        # print("trX:",trX)
        # print("trY:",trY)
        # print("trX.shape[0]:",trX.shape[0])
        # print("hidden layer",self.hidden_layer_size)       
        # エラーになるのでいったんはずす
        # summary_str = sess.run(summary, feed_dict={ input_data: trX,
        #                                             actual_labels: trY,
        #                                             initial_state: np.zeros([trX.shape[0], self.hidden_layer_size])})
        

        # summary_writer.add_summary(summary_str, epoch)
        # summary_writer.flush()

      #学習したモデルを保存
      saver = tf.compat.v1.train.Saver()
      saver.save(sess, self.model_filename)

      #最後にテストデータでの精度を表示
      a = sess.run(accuracy, feed_dict={input_data: teX,
                                        actual_labels: teY,
                                        initial_state: np.zeros([teX.shape[0], self.hidden_layer_size])})
      print("Accucacy on test=",a)


  def predict(self, seq):
    print("predict")
    """
    文章を入力したときに次に来る単語を予測する  
    param seq:予測したい単語の直前の文字列 chunk_size 以上の単語が必要

    """
    #最初に復元したい変数をすべて定義
    tf.compat.v1.reset_default_graph()
    input_data = tf.compat.v1.placeholder("float", [None, self.chunk_size, self.input_layer_size])
    initial_state = tf.compat.v1.placeholder("float", [None, self.hidden_layer_size])
    prediction = tf.nn.softmax(self.inference(input_data, initial_state))
    predicted_labels = tf.argmax(prediction, 1)

    #入力データの作成
    #seqをone-hot表現に変換

    words = [word for word in seq.split() if not word.startswith("-")]
    x = np.zeros([1, self.chunk_size, self.input_layer_size])
    for i in range(self.chunk_size):
      word = seq[len(words)-self.chunk_size+i]
      index = self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])
      x[0][i][index] = 1
    feed_dict = {
        input_data: x, #(1, chunk_size, vocabulary_size)
        initial_state: np.zeros([1, self.hidden_layer_size])
    }

    # tf.Session()を容易
    with tf.compat.v1.Session() as sess:
      #保存したモデルをロード
      #ロード前にすべての変数を用意しておく必要あり
      saver = tf.compat.v1.train.Saver()
      saver.restore(sess, self.model_filename)

      #ロードしたモデルを使って予測結果を計算
      u, v = sess.run([prediction, predicted_labels], feed_dict=feed_dict)

      keys = list(self.dictionary.keys())

      #コンソールに文字ごとの確率を表示
      
      cols = ['word', 'result']
      df = pd.DataFrame(index=[], columns=cols)
      
      for i in range(self.vocabulary_size):
        c = self.unknown_word_symbol if i == (self.vocabulary_size -1 ) else keys[i]
        # print(c, ":", u[0][i])
        record = pd.Series([c, u[0][i]], index=df.columns)
        df = df.append(record, ignore_index=True)

      #上位10位まで表示

      print("Predict:", seq + " " + ("<???>" if v[0]==(self.vocabulary_size -1) else keys[v[0]]))

    return u[0], df


# 実行

## 学習

In [ ]:
In = Language()

In.train()

init
有効な単語数= 9826
総単語数= 34010
うち、未知の単語数= 24185
有効な単語数(unknown symbol含む)= 9826
pickle保存済
{'jet': 0, 'lag': 1, 'and': 2, 'the': 3, 'monday': 4, 'of': 5, 'my': 6, 'life': 7, 'i': 8, 'recently': 9, 'returned': 10, 'to': 11, 'us': 12, 'from': 13, 'australia': 14, 'flight': 15, 'took': 16, 'me': 17, 'morning': 18, 'in': 19, 'sydney': 20, 'again': 21, 'la': 22, 'crossing': 23, 'date': 24, 'line': 25, 'messed': 26, 'up': 27, 'sense': 28, 'time': 29, 'enough': 30, 'without': 31, 'added': 32, 'bonus': 33, 'thinking': 34, 'should': 35, 'be': 36, 'heading': 37, 'bed': 38, 'just': 39, 'as': 40, 'sun': 41, 'began': 42, 'climb': 43, 'into': 44, 'california': 45, 'sky': 46, 'you': 47, 'may': 48, 'familiar': 49, 'with': 50, 'concept': 51, 'name': 52, 'for': 53, 'circadian': 54, 'disruption': 55, 'sleep': 56, 'cycles': 57, 'rhythms': 58, 'if': 59, "you've": 60, 'had': 61, 'pleasure': 62, 'zones': 63, 'a': 64, 'plane': 65, 'whether': 66, 'it': 67, 'was': 68, 'mere': 69, 'hop': 70, 'one': 71, 'coast': 72,

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:421: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


prepare_data
Epoch 0 completed out of 1000 -- loss: 9.192710876464844 -- accuracy: 0.0
Epoch 1 completed out of 1000 -- loss: 9.158159255981445 -- accuracy: 0.0
Epoch 2 completed out of 1000 -- loss: 9.121472358703613 -- accuracy: 0.00390625
Epoch 3 completed out of 1000 -- loss: 9.081955909729004 -- accuracy: 0.00390625
Epoch 4 completed out of 1000 -- loss: 9.03948974609375 -- accuracy: 0.00390625
Epoch 5 completed out of 1000 -- loss: 8.994088172912598 -- accuracy: 0.00390625
Epoch 6 completed out of 1000 -- loss: 8.945845603942871 -- accuracy: 0.00390625
Epoch 7 completed out of 1000 -- loss: 8.8948974609375 -- accuracy: 0.00390625
Epoch 8 completed out of 1000 -- loss: 8.841407775878906 -- accuracy: 0.00390625
Epoch 9 completed out of 1000 -- loss: 8.785565376281738 -- accuracy: 0.00390625
Epoch 10 completed out of 1000 -- loss: 8.727575302124023 -- accuracy: 0.00390625
Epoch 11 completed out of 1000 -- loss: 8.66765308380127 -- accuracy: 0.00390625
Epoch 12 completed out of 1000 

## 予測

In [ ]:
_, df = In.predict("some of them looks like")

predict
inference
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/RabbitChallenge/DNN_code_colab_lesson_3_4/lesson_3/3_2_tf_languagemodel/data_for_predict/predict_model.ckpt


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:421: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Predict: some of them looks like i


In [ ]:
df = df.sort_values('result',ascending=False)

df

,word,result
8,i,4.658158e-02
314,they,2.912564e-02
64,a,2.719134e-02
11,to,2.524583e-02
93,that,2.330689e-02
...,...,...
8728,crewmen,4.094109e-07
6025,wood,4.082202e-07
8380,violated,4.070998e-07
4961,largest,4.036021e-07


# 終わり

In [ ]:
#終了時刻
finish_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print("finish_time=",finish_time)
print("total_time=",finish_time-start_time)


finish_time= 2021-05-05 21:54:49.856583+09:00
total_time= 0:13:41.680947
